In [ ]:
import pandas as pd
import requests
import time
import csv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Load imbd movies dataset
url = '/Users/rishikeshdhokare/Downloads/MergedDataSet/Cleaned/final_merged_file.csv'
df = pd.read_csv(url)
######### Year column #########
df['Year_y'] = np.where(df['Year_y'].isnull(), df['Year_x'], df['Year_y'])
df.drop('Year_x', axis=1, inplace=True)

######### Year column #########
# Merge Genre_x and Genre_y into a new column called 'Genre'
df['Genre'] = df['Genre_x'].fillna('') + df['Genre_y'].fillna('')
df['Genre'] = df['Genre'].str.strip()

df.drop('Genre_x', axis=1, inplace=True)
df.drop('Genre_y', axis=1, inplace=True)

######### Metascore column #########
df['Metascore_y'] = df['Metascore_y'].fillna(df['Metascore_x'])
df.drop(columns=['Metascore_x'], inplace=True)
df.rename(columns={'Metascore_y': 'Metascore'}, inplace=True)

######### Metascore column #########
df['Director_y'] = df['Director_y'].fillna(df['Director_x'])
df.drop(columns=['Director_x'], inplace=True)
df.rename(columns={'Director_y': 'Director'}, inplace=True)

df.columns = df.columns.str.replace(r'(_x_x|_x_y|_x|_y)$', '', regex=True)

######### imdbRating column #########
# Replace null values in 'imdbRating' with values from 'Rating'
df['imdbRating'] = df['imdbRating'].fillna(df['Rating'])
df.drop(columns=['Rating'], inplace=True)

######### Cast column #########
df['Cast'] = df['Cast'].fillna(df['Actors'])
df.drop('Actors', axis=1, inplace=True)

# Drop rows where 'BoxOffice' is NaN or empty
df = df.dropna(subset=['BoxOffice'])
df = df[df['BoxOffice'] != '']

In [ ]:
# Step 3: Clean up the genre column (remove spaces if necessary)
df['Genre'] = df['Genre'].str.strip()
df['Genre']

# Step 2: Split the genres into separate rows for movies with multiple genres
df = df.assign(Genre = df['Genre'].str.split(',')).explode('Genre')

# Step 4: Convert BoxOffice and imdbVotes to numeric (after removing non-numeric characters)
df['BoxOffice'] = pd.to_numeric(df['BoxOffice'].replace(r'[\$,]', '', regex=True), errors='coerce')
df['imdbVotes'] = pd.to_numeric(df['imdbVotes'].replace(',', '', regex=True), errors='coerce')

# Step 5: Group by Genre and calculate average IMDb rating, vote count, and box office earnings
genre_statistics_df = df.groupby('Genre').agg(
    avg_rating=('imdbRating', 'mean'),
    avg_votes=('imdbVotes', 'mean'),
    avg_box_office=('BoxOffice', 'mean')
).reset_index()

# Step 6: Sort the genres by average rating and votes
genre_statistics_df = genre_statistics_df.sort_values(by=['avg_rating', 'avg_votes'], ascending=False)


In [ ]:
# Table 1: Sorted by average rating and vote count (already sorted in previous steps)
print("Table 1: Sorted by Average Rating and Vote Count")
genre_statistics_df.round(2)

In [ ]:
# Plot 1: Average IMDb Ratings by Genre
plt.figure(figsize=(8, 5))
sns.barplot(x='avg_rating', y='Genre', data=genre_stats)
plt.title('Average IMDb Ratings by Genre')
plt.show()

In [ ]:
# Table 3: Sorted by avg_box_office in descending order
genre_statistics_df['avg_box_office'] = genre_statistics_df['avg_box_office'].astype(str)
genre_statistics_df['avg_box_office'] = genre_statistics_df['avg_box_office'].replace({'\$': '', ',': ''}, regex=True).astype(float)
genre_stats_by_box_office = genre_statistics_df.sort_values(by='avg_box_office', ascending=False)
print("\nTable 3: Sorted by Average Box Office")
genre_stats_by_box_office.round(2)

In [ ]:
# Table 3: Sorted by avg_box_office in descending order
genre_stats_by_box_office = genre_stats.sort_values(by='avg_box_office', ascending=False, key=lambda x: x.str.replace('[$,]', '', regex=True).astype(float))
print("\nTable 3: Sorted by Average Box Office")
genre_stats_by_box_office.round(2)

In [ ]:
# Plot 2: Average Votes by Genre
plt.figure(figsize=(8, 5))
sns.barplot(x='avg_votes', y='Genre', data=genre_stats_by_votes)
plt.title('Average Votes by Genre')
plt.show()

In [ ]:
# Plot 3: Average Box Office by Genre
plt.figure(figsize=(8, 5))
sns.barplot(x='avg_box_office_numeric', y='Genre', data=genre_stats)
plt.title('Average Box Office by Genre')
plt.show()